## Applied Data Science Capstone Week 3: Peer-graded Assignment (Part 1)

### (scroll down for Parts 2 & 3)

In [85]:
import pandas as pd # pandas library
!conda install -c conda-forge lxml --yes
from lxml import etree
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [94]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# obtain table from url and load in pandas dataframe
df = pd.read_html(url, header=0)[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [95]:
# remove all rows with un-assigned Burough
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [96]:
df = df.rename(columns={"Neighbourhood": "Neighborhood"})
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [97]:
# combine all neighborhoods with the same postal code area
df.groupby(['Postal Code'])
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [98]:
# check for any un-assigned neighborhood
d = df.loc[df['Neighborhood'] == 'Not assigned']
d

,Postal Code,Borough,Neighborhood


In [99]:
# none found, so no additional coding needed to replace un-assigned neighborhood with burough

In [100]:
# clean with re-index
df = df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [101]:
df.shape

(103, 3)

## Applied Data Science Capstone Week 3: Peer-graded Assignment (Part 2)

In [102]:
# load geographical coordinates of the neighborhoods in df2 dataframe
df2 = pd.read_csv('https://cocl.us/Geospatial_data')
df2

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [103]:
# merge dataframes based on same Postal Code
df3 = pd.merge(df, df2, on="Postal Code")
df3

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Applied Data Science Capstone Week 3: Peer-graded Assignment (Part 3)

In [109]:
# google search for geolocator coordinates for Toronto, Canada yields latitude 43.651070 longitude -79.347015
# create cluster map neighborhoods in Toronto using latitude and longitude values

map_toronto = folium.Map(location=[43.651070, -79.347015], zoom_start=10)

# add markers to map
for lat, lng, postalcode, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Postal Code'], df3['Borough'], df3['Neighborhood']):
    label = '{}, {}'.format(postalcode, neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [111]:
# Generated map shows higher density cluster of unique Postal Code around the Toronto city center

In [110]:
# create new dataframe of boroughs that contain the word Toronto
df4 = df3.loc[df3['Borough'].str.contains("Toronto")]
df4 = df4.reset_index(drop=True)
df4

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [112]:
# create new cluster map of neighborhoods with boroughs that contain the word Toronto
map_toronto2 = folium.Map(location=[43.651070, -79.347015], zoom_start=12)

# add markers to map
for lat, lng, postalcode, borough, neighborhood in zip(df4['Latitude'], df4['Longitude'], df4['Postal Code'], df4['Borough'], df4['Neighborhood']):
    label = '{}, {}'.format(postalcode, neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto2)  
    
map_toronto2

In [ ]:
# This map likewise shows higher density cluster of unique Postal Code around the Toronto city center